In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
cd PatientTrajectoryForecasting

[Errno 2] No such file or directory: 'PatientTrajectoryForecasting'
/home/sifal.klioui/PatientTrajectoryForecasting


In [17]:
from utils.data_preparation import load_mimic_data, clean_data, icd_mapping, trim, build_data, remove_code, save_files, generate_code_types, filter_subjects, filter_notes
from utils.utils import get_paths, store_files
import yaml 
import os
import pandas as pd

In [18]:
with open('paths.yaml', 'r') as file:
    path_config = yaml.safe_load(file)

In [19]:
paths = get_paths(path_config) 

In [21]:
subject_id_adm_map, adm_dx_map, adm_px_map, adm_drug_map, drug_description_map, notes = load_mimic_data(**paths)

Building subject_id-admission mapping, admission-date mapping
Building admission-diagnosis mapping
-Number of null ICD9 codes in file /home/sifal.klioui/mimic-iv-2.2/hosp/diagnoses_icd.csv.gz: 0
-Number of null ICD10 codes in file /home/sifal.klioui/mimic-iv-2.2/hosp/diagnoses_icd.csv.gz: 0
Building admission-procedure mapping
-Number of null ICD9 codes in file /home/sifal.klioui/mimic-iv-2.2/hosp/procedures_icd.csv.gz: 0
-Number of null ICD10 codes in file /home/sifal.klioui/mimic-iv-2.2/hosp/procedures_icd.csv.gz: 0
Building admission-drug mapping


In [26]:
min_visits = 2
subject_id_adm_map, adm_dx_map, adm_px_map, adm_drug_map = clean_data(subject_id_adm_map, adm_dx_map, adm_px_map, adm_drug_map, min_visits)

Cleaning data...
Removing patient records who do not have all three medical codes for an admission
Removing patients who made less than 2 admissions
 Total Number of patients 18383
 Total Number of admissions 48855
 Average number of admissions per patient 2.6576184518304955
 Total Number of diagnosis code 751
 Total Number of procedure code 466
 Total Number of drug code 4839
 Total Number of codes 6056
 average Number of procedure code per visit 3.05
 average Number of diagnosis code per visit 12.49
 average Number of Drug code per visit 51.027366697369764


In [27]:
print(len(subject_id_adm_map.keys()))
print(max(subject_id_adm_map.keys()))

18383
19999840


In [28]:
adm_dx_map, adm_px_map, code_description_map = icd_mapping(adDx = adm_dx_map, adPx = adm_px_map, adDrug = adm_drug_map, drugDescription = drug_description_map, **paths)

Total ICD to CCS entries: 144424
Total CCS codes/descriptions: 514
Total number of unique codes (Diag + Proc): 828
total number of ICD9 codes used 0 and ICD10 codes: 0
-Total number (complete set) of ICD9+ICD10 codes (diag + proc): 144424
-Total number of ICD codes actually used: 14
-Total number of ICD codes missing in the admissions list:  576118
total number of ICD9 codes used 0 and ICD10 codes: 0
-Total number (complete set) of ICD9+ICD10 codes (diag + proc): 144424
-Total number of ICD codes actually used: 33
-Total number of ICD codes missing in the admissions list:  139599
 Total Number of diagnosis code 10
 Total Number of procedure code 8
 Total Number of drug code 4839
 Total Number of unique  D,P codes 18
 Total Number of all codes 4857
 average Number of procedure code per visit 1.33
 average Number of diagnosis code per visit 1.80
 average Number of drug code per visit 51.03
 Min. and max. Number of diagnosis code per admission (1, 10)
 Min. and max. Number of procedure co

In [29]:
print(len(adm_dx_map.keys()))
print(max(adm_dx_map.keys())) 

18863
29999670


In [30]:
max_dx, max_px, max_drg = 80, 80, 80 
adm_dx_map, adm_px_map, adm_drug_map = trim(adm_dx_map, adm_px_map, adm_drug_map, max_dx, max_px, max_drg)

Trimming the diagnosis, procedure, and medication codes for each visit
 Total Number of diagnosis code 10
 Total Number of procedure code 8
 Total Number of drug code 4730
 Total Number of unique  D,P codes 18
 Total Number of all codes 4748
 average Number of procedure code per visit 1.33
 average Number of diagnosis code per visit 1.80
 average Number of drug code per visit 38.71
 Min. and max. Number of diagnosis code per admission (1, 10)
 Min. and max. Number of procedure code  per admission(1, 11)
 Min. and max. Number of drug code  per admission (1, 80)


In [31]:
print(len(adm_dx_map.keys()))
print(max(adm_dx_map.keys())) 

18863
29999670


In [32]:
subject_id_adm_map = filter_subjects(subject_id_adm_map, min_visits)

Number of patients with less than 2 visits: 0


In [35]:
sum(len(x) for x in subject_id_adm_map.values())

48855

In [33]:
filted_notes, subject_id_hadm_id_map_ = filter_notes(notes, subject_id_adm_map, min_visits)

filtering notes where the subject has made less than 2 visits sucessive visits...
found 5280 subjects and 13206 visits that need to be removed


In [ ]:
patients_visits_sequences, tokens_ids_map = build_data(subject_id_adm_map, adm_dx_map, adm_px_map, adm_drug_map)

In [ ]:
min_freq_codes = 5
patients_visits_sequences, tokens_ids_map, ids_tokens_map  = remove_code(patients_visits_sequences, tokens_ids_map, threshold = min_freq_codes)

In [ ]:
save_files(patients_visits_sequences, dict(tokens_ids_map), code_description_map)

In [ ]:
generate_code_types(ids_tokens_map)